# 1. Load modules

In [72]:
import arcpy

# 2. Define functions for symbology

In [80]:
def update_symbology_modes(date, airport_name, select_runway_area):
    
    if select_runway_area == True:
        layer_name = "MODES_{0}_XYTableToPoint2_{1}_selection".format(date, airport_name)
    if select_runway_area == False:
        layer_name = "MODES_{0}_XYTableToPoint2_{1}".format(date, airport_name)
    
    p = arcpy.mp.ArcGISProject("CURRENT")
    m = p.listMaps("Map")[0]
    #lyr = m.listLayers("MODES_{0}_XYTableToPoint2_{1}_selection".format(date, airport_name))[0]
    lyr = m.listLayers(layer_name)[0]
    sym = lyr.symbology
    sym.updateRenderer('GraduatedColorsRenderer')
    sym.renderer.classificationField = "GNSS_ALTD"
    colorRamp = p.listColorRamps("Blues (Continuous)")[0]
    sym.renderer.colorRamp = colorRamp

    # using natural breaks
    #sym.classificationMethod = 'NaturalBreaks'
    #sym.renderer.breakCount = 10

    # using manual interval
    sym.classificationMethod = 'ManualInterval'
    #classBreakValues = [100,200,500,1000,2000,5000,10000,15000]
    #classBreakLabels = ['<100','<200','<500','<1000','<2000','<5000','<10000','<15000']
    classBreakValues = [100,200,500,1000,1500]
    classBreakLabels = ['<100','<200','<500','<1000','<1500']
    
    sym.renderer.breakCount = len(classBreakValues)

    count = 0
    for brk in sym.renderer.classBreaks:
        brk.upperBound = classBreakValues[count]
        brk.label = classBreakLabels[count]
        count += 1

    lyr.symbology = sym
    return(sym)

In [77]:
def update_symbology_amdar(date, airport_name, select_runway_area):
    
    if select_runway_area == True:
        layer_name = "AMDARS_{0}_XYTableToPoint2_{1}_selection".format(date, airport_name)
    if select_runway_area == False:
        layer_name = "AMDARS_{0}_XYTableToPoint2_{1}".format(date, airport_name)
    
    p = arcpy.mp.ArcGISProject("CURRENT")
    m = p.listMaps("Map")[0]
    #lyr = m.listLayers("AMDARS_{0}_XYTableToPoint2_{1}_selection".format(date, airport_name))[0]
    lyr = m.listLayers(layer_name)[0]
    sym = lyr.symbology
    sym.updateRenderer('GraduatedColorsRenderer')
    sym.renderer.classificationField = "ALTD"
    colorRamp = p.listColorRamps("Reds (Continuous)")[0]
    sym.renderer.colorRamp = colorRamp

    # using natural breaks
    #sym.classificationMethod = 'NaturalBreaks'
    #sym.renderer.breakCount = 10

    # using manual interval
    sym.classificationMethod = 'ManualInterval'
    #classBreakValues = [100,200,500,1000,2000,5000,10000,15000]
    #classBreakLabels = ['<100','<200','<500','<1000','<2000','<5000','<10000','<15000']
    classBreakValues = [100,200,500,1000,1500]
    classBreakLabels = ['<100','<200','<500','<1000','<1500']
    sym.renderer.breakCount = len(classBreakValues)

    count = 0
    for brk in sym.renderer.classBreaks:
        brk.upperBound = classBreakValues[count]
        brk.label = classBreakLabels[count]
        count += 1

    lyr.symbology = sym
    return(sym)

# 3. Inputs

In [84]:
date_list = [20220919]

airport_name_list = ['Cardiff']

'''
airport_name_list = ['Heathrow', \
                     'Gatwick', \
                     'Manchester', \
                     'Stansted', \
                     'Edinburgh', \
                     'Birmingham', \
                     'Bristol', \
                     'Glasgow', \
                     'Aberdeen', \
                     'EastMidlands', \
                     'LondonCity', \
                     'BelfastInt', \
                     'Newcastle', \
                     'LeedsBradford', \
                     'Liverpool',\
                     'Cardiff']
'''

select_runway_area = False # Set to True or False - defines whether runway polygons are used to subset the data

# 4. Main code for loading

In [79]:
for airport_name in airport_name_list:
    for date in date_list:
        
        print('Loading data for: {0} {1}'.format(airport_name, date))
        
        arcpy.management.XYTableToPoint(r"D:\MetDB\AMDARS\{0}\AMDARS_{1}.txt".format(airport_name,date), 
                                    r"C:\Users\gemma.daron\OneDrive - Met Office\ArcGISProProjects\ModeS\ModeS.gdb\AMDARS_{0}_XYTableToPoint2_{1}".format(date, airport_name), 
                                    "LON", 
                                    "LAT", 
                                    None, 
                                    'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
    
        arcpy.management.XYTableToPoint(r"D:\MetDB\Mode-S\{0}\MODE-S_{1}.txt".format(airport_name,date), 
                                    r"C:\Users\gemma.daron\OneDrive - Met Office\ArcGISProProjects\ModeS\ModeS.gdb\MODES_{0}_XYTableToPoint2_{1}".format(date, airport_name), 
                                    "LON", 
                                    "LAT", 
                                    None, 
                                    'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

        # Select features within runway polygons and delete  (ISSUE WITH AMDAR DATA NOT ALIGNING WITH RUNWAY)
        if select_runway_area == True:

            arcpy.SelectLayerByLocation_management ("AMDARS_{0}_XYTableToPoint2_{1}".format(date, airport_name),
                                              "WITHIN",
                                              "{0}_buffer".format(airport_name))
            arcpy.CopyFeatures_management("AMDARS_{0}_XYTableToPoint2_{1}".format(date, airport_name), "AMDARS_{0}_XYTableToPoint2_{1}_selection".format(date, airport_name))
            arcpy.Delete_management("AMDARS_{0}_XYTableToPoint2_{1}".format(date, airport_name))
           
    
            arcpy.SelectLayerByLocation_management ("MODES_{0}_XYTableToPoint2_{1}".format(date, airport_name),
                                              "WITHIN",
                                              "{0}_buffer".format(airport_name))
            arcpy.CopyFeatures_management("MODES_{0}_XYTableToPoint2_{1}".format(date, airport_name), "MODES_{0}_XYTableToPoint2_{1}_selection".format(date, airport_name))
            arcpy.Delete_management("MODES_{0}_XYTableToPoint2_{1}".format(date, airport_name))
    
        
        # Update symbology of loaded layers
        update_symbology_amdar(date, airport_name, select_runway_area)
        update_symbology_modes(date, airport_name, select_runway_area)

Loading data for: Cardiff 20220919
